In [2]:
from datasets import load_dataset
import json

# 加载SQuAD数据集
squad = load_dataset("squad")

# 使用字典来存储首次出现的context和对应的完整example
unique_examples = {}

for example in squad['train']:
    context = example['context']
    if context not in unique_examples:
        unique_examples[context] = example

# 输出不同context的数量
print(f"Total number of unique contexts: {len(unique_examples)}")

# 将这些唯一的examples保存到JSON文件中
with open('unique_contexts.json', 'w') as f:
    json.dump(list(unique_examples.values()), f, indent=4, ensure_ascii=False)


/home/yuhao/anaconda/envs/easycontext/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating validation split: 100%|██████████| 10570/10570 [00:00<00:00, 668735.10 examples/s]


Total number of unique contexts: 18891


In [ ]:
import random
import string

def generate_unique_string(length=12, seen=set()):
    """
    Generate a unique string of specified length that includes at least one lowercase letter, one uppercase letter,
    one digit, and one special character from the set {@, $, %, #, &}.
    Ensures that no string is repeated using a 'seen' set.
    """
    # Define the character pools
    lower = string.ascii_lowercase
    upper = string.ascii_uppercase
    digits = string.digits
    special = "@$%#&"
    
    while True:
        # Ensure at least one character from each required category
        characters = [
            random.choice(lower),
            random.choice(upper),
            random.choice(digits),
            random.choice(special)
        ]
        
        # Fill the rest of the string with a random choice from the combined pool
        combined_pool = lower + upper + digits + special
        characters.extend(random.choices(combined_pool, k=length - 4))
        
        # Shuffle to ensure the characters are not in the same order every time
        random.shuffle(characters)
        
        # Join the list into a string
        result = ''.join(characters)
        
        # Check if this result has been generated before
        if result not in seen:
            seen.add(result)
            return result

# Example usage
seen_strings = set()
for _ in range(10):  # Generate 10 unique strings
    print(generate_unique_string(seen=seen_strings))


In [ ]:
import json
import os



def merge_json_files(folder_path, output_file):
    """ Merges multiple JSON files from a specified folder into a single JSON file """
    merged_data = []
    
    # List all files in the given folder
    files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
    files.sort()  # Optional, sorts the files by name if needed for ordered merging
    
    # Iterate over each file and append its content to merged_data
    for file in files:
        file_path = os.path.join(folder_path, file)
        with open(file_path, 'r') as infile:
            data = json.load(infile)
            merged_data.extend(data)  # Append the data from each file to the list
    
    # Write the merged data to a new JSON file
    with open(output_file, 'w') as outfile:
        json.dump(merged_data, outfile, indent=4)
    
    print(f"All data has been merged into {output_file}.")

# Usage
folder_path = 'merge_data'  # Folder containing the JSON files
output_file = 'merged_data.json'  # Name of the output file
merge_json_files(folder_path, output_file)


In [ ]:
import json

data_index = 0

def count_elements_in_json(filepath):
    """
    Loads a JSON file and counts the number of top-level elements.
    """
    try:
        with open(filepath, 'r') as file:
            data = json.load(file)
            if isinstance(data, dict):
                # If the top level is a dictionary, count the number of keys
                return len(data)
            elif isinstance(data, list):
                # If the top level is a list, count the number of items
                return len(data)
            else:
                # Other JSON structures are not expected
                return 0
    except Exception as e:
        print(f"Error loading JSON data from {filepath}: {e}")
        return None

# Example usage
filepath = 'version_10_test_4k.json'  # Replace with your actual file path
element_count = count_elements_in_json(filepath)
if element_count is not None:
    print(f"The number of top-level elements in the JSON file is: {element_count}")


In [ ]:

import json
import os
import random
from hashlib import sha256
from datasets import load_dataset

global data_index
dataset = load_dataset("DKYoon/SlimPajama-6B", split='train')


In [ ]:



def generate_unique_string():
    """ Generate a unique string based on hashing """
    unique_id = sha256(os.urandom(48)).hexdigest()[:16]
    return unique_id

import random
def modify_gpt_answer(gpt_answer, correct_answer_id, question_index):
    """ Modifies the GPT_answer to include the correct answer ID at the appropriate position """
    insert_text = f"\n\n\n### Retrieval: The answer to question {question_index} can be found in ID {correct_answer_id}.\n\n"
    analysis_index = gpt_answer.find("### Analysis:")
    if analysis_index != -1:
        part_before = gpt_answer[:analysis_index + len("### Analysis:")]
        part_after = gpt_answer[analysis_index + len("### Analysis:"):]
        return  insert_text + part_before  + part_after
    else:
        return  insert_text  +  gpt_answer 




def add_ctxs_to_item(base_item, items, data_index):
    """ Adds 'ctxs' key to a group of items and updates GPT_answer """
    ctxs = []
    questions = []
    questions_ctx = []
    gpt_answers = []
    answer_map = {}
    # Process each item to create questions, answers, and modify GPT_answer
    for idx, item in enumerate(items):
        context_id = generate_unique_string()
        questions.append(f"Question {idx + 1}: {item['question']}")
        if idx <= 2:
            question_id = generate_unique_string()
            ctxs.append({
                "ID": question_id,
                "text": item['question'],
                "hasAnswer": 'question',
                "GPT_answer":item['GPT_answer'],
                "context_id": context_id
            })
            ctxs.append({
                "ID": context_id,
                "text": item['context'],
                "hasAnswer": True
            })
            # Store answer mapping for modification later
        else:
            ctxs.append({
                "ID": context_id,
                "text": item['context'],
                "hasAnswer": False
            })
    # Add random contexts from dataset
    data_index = add_contexts(dataset, ctxs, data_index)

    # Shuffle the list of contexts to ensure randomness
    random.shuffle(ctxs)
    # Reconstruct the mapping of question IDs to answer IDs after shuffling

    for ctx in ctxs:
        if ctx['hasAnswer'] == 'question':
            gpt_answers.append(modify_gpt_answer(ctx['GPT_answer'],ctx['context_id'],ctx['ID']))
            

    base_item['GPT_answer'] = ' '.join(gpt_answers)
    base_item['question'] = ' '.join(questions)
    return ctxs, data_index


def add_contexts(dataset, ctxs, data_index):
    """ Adds contexts from the dataset while checking the token conditions """
    total_tokens = 0
    max_tokens=12500
    while total_tokens < max_tokens:
        text = dataset[data_index]['text']
        data_index = data_index+1
        word_count = len(text.split())
        if 80 <= word_count <= 500:
            context_id = generate_unique_string()
            ctxs.append({"ID": context_id, "text": text, "hasAnswer": False})
            total_tokens += word_count  # Update the token count assuming 1 token per word
    return data_index





In [ ]:
def load_json_data(filepath):
    """ Loads JSON data from a file """
    try:
        with open(filepath, 'r') as file:
            data = json.load(file)
            return data
    except Exception as e:
        print(f"Error loading JSON data from {filepath}: {e}")
        return []
    
def expand_and_enhance_data(data, data_index ):
    """ Expands each item by 32 times and enhances with ctxs """
    enhanced_data = []
    for item in data:
        for _ in range(1):  # Duplicate each item 32 times
            # Select 8 other random items
            other_items = random.sample(data, 19)
            other_items.insert(0, item)  # Include the current item as the first item
            new_item = item.copy()  # Make a copy of the item
            new_item['ctxs'],data_index = add_ctxs_to_item(new_item, other_items,data_index)
            enhanced_data.append(new_item)
    random.shuffle(enhanced_data)  # Shuffle all items to mix them up
    return enhanced_data, data_index

def save_data(data, output_file):
    """ Saves processed data into a JSON file """
    with open(output_file, 'w') as outfile:
        json.dump(data, outfile, indent=4)
    print(f"Data saved to {output_file}")

# Main logic
merged_file_path = 'merged_data.json'
output_file = 'version_6_test_4k.json'
merged_data = load_json_data(merged_file_path)
merged_data = merged_data[20:4020]

if merged_data:
    data_index = 0
    enhanced_data,data_index = expand_and_enhance_data(merged_data,data_index )
    save_data(enhanced_data, output_file)
    print(data_index)
else:
    print("Failed to load data or context data.")

In [ ]:
import json
import os
import random
from hashlib import sha256

def generate_unique_string():
    """ Generate a unique string based on hashing """
    unique_id = sha256(os.urandom(48)).hexdigest()[:24]
    return unique_id

def load_json_data(filepath):
    """ Loads JSON data from a file """
    try:
        with open(filepath, 'r') as file:
            data = json.load(file)
            return data
    except Exception as e:
        print(f"Error loading JSON data from {filepath}: {e}")
        return []

def modify_gpt_answer(gpt_answer, correct_answer_id, question_index):
    """ Modifies the GPT_answer to include the correct answer ID at the appropriate position """
    insert_text = f"\n\n### Retrieval: The answer to question {question_index + 1} can be found in ID {correct_answer_id}."
    analysis_index = gpt_answer.find("### Analysis:")
    if analysis_index != -1:
        part_before = gpt_answer[:analysis_index + len("### Analysis:")]
        part_after = gpt_answer[analysis_index + len("### Analysis:"):]
        return  insert_text + part_before  + part_after
    else:
        return  insert_text  + gpt_answer 

def add_ctxs_to_item(base_item, items, context_data):
    """ Adds 'ctxs' key to a group of items and updates GPT_answer """
    ctxs = []
    questions = []

    gpt_answers = []

    # Process each item to create questions, answers, and modify GPT_answer
    for idx, item in enumerate(items):
        # Generate a unique ID for each context, all will have answers
        context_id = generate_unique_string()
        questions.append(f"Question {idx + 1}: {item['question']}")
        
        gpt_answers.append(modify_gpt_answer(item['GPT_answer'], context_id, idx))
        # Add current item's context with hasAnswer=True
        ctxs.append({
            "ID": context_id,
            "text": item['context'],
            "hasAnswer": True
        })

    # Add 80 random contexts from context_data without answers
    sampled_contexts = random.sample(context_data, 90)
    for context in sampled_contexts:
        ctxs.append({
            "ID": generate_unique_string(),
            "text": context['context'],
            "hasAnswer": False
        })

    # Shuffle the list of contexts to ensure randomness
    random.shuffle(ctxs)
    # Update the base item
    base_item['question'] = ' '.join(questions)
    base_item['GPT_answer'] = ' '.join(gpt_answers)
    return ctxs


def expand_and_enhance_data(data, context_data):
    """ Expands each item by 32 times and enhances with ctxs """
    enhanced_data = []
    for item in data:
        for _ in range(15):  # Duplicate each item 32 times
            # Select 8 other random items
            other_items = random.sample(data, 9)
            other_items.insert(0, item)  # Include the current item as the first item
            new_item = item.copy()  # Make a copy of the item
            new_item['ctxs'] = add_ctxs_to_item(new_item, other_items, context_data)
            enhanced_data.append(new_item)
    random.shuffle(enhanced_data)  # Shuffle all items to mix them up
    return enhanced_data

def save_data(data, output_file):
    """ Saves processed data into a JSON file """
    with open(output_file, 'w') as outfile:
        json.dump(data, outfile, indent=4)
    print(f"Data saved to {output_file}")

# Main logic
merged_file_path = 'merged_data.json'
context_file_path = 'unique_contexts.json'
output_file = 'enhanced_data.json'

merged_data = load_json_data(merged_file_path)
context_data = load_json_data(context_file_path)
merged_data = merged_data
context_data = context_data[1000:]
if merged_data and context_data:
    enhanced_data = expand_and_enhance_data(merged_data, context_data)
    save_data(enhanced_data, output_file)
else:
    print("Failed to load data or context data.")


In [ ]:
#变为行
import json

# 打开并读取JSON文件
with open('version_6_test_4k.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 检查data是否是列表类型，如果是，按行写入每个元素
if isinstance(data, list):
    with open('line_to_line.json', 'w', encoding='utf-8') as output:
        for item in data:
            json.dump(item, output)
            output.write('\n')
else:
    print("JSON root is not a list. Cannot save items line by line.")


# Version6

In [ ]:
import json
import os
import random
from hashlib import sha256
from datasets import load_dataset

dataset_1 = load_dataset("Open-Orca/OpenOrca")

dataset_2 = load_dataset("DKYoon/SlimPajama-6B", split='train')


In [ ]:


def filter_criteria(example):
    # 计算单词数量
    question_words = len(example['question'].split())
    response_words = len(example['response'].split())
    
    # 将question转换为小写并检查是否含有单词"translate"或"translation"
    # question_lower = example['question'].lower()
    # contains_translate = "translate" in question_lower or "translation" in question_lower
    id_starts_with_cot = example['id'].startswith('cot')
    # 判断单词数量是否都小于1000且question中不含"translate"和"translation"
    # return 80 < question_words < 1000 and 80 < response_words < 1000 and not contains_translate and id_starts_with_cot
    return  id_starts_with_cot and 20 < response_words and 20 < question_words

# 应用过滤条件
dataset_1 = dataset_1.filter(filter_criteria)

# 随机打乱数据集
dataset_1= dataset_1.shuffle()
dataset_2 = dataset_2.shuffle()
# 取出训练集
filtered_dataset = dataset_1['train']
dataset_1 = filtered_dataset
# 检查过滤后的数据
print(filtered_dataset)


In [ ]:

def modify_gpt_answer_end(gpt_answer, correct_answer_id, question_index):
    """ Modifies the GPT_answer to include the correct answer ID at the appropriate position """
    insert_text = f"### Retrieval: The answer to question {question_index} can be found in ID {correct_answer_id}."
    analysis_index = gpt_answer.find("### Analysis:")
    if analysis_index != -1:
        part_before = gpt_answer[:analysis_index + len("### Analysis:")]
        part_after = gpt_answer[analysis_index + len("### Analysis:"):]
        return  insert_text + part_before  + part_after
    else:
        return  insert_text  + gpt_answer 

def generate_unique_string():
    """ Generate a unique string based on hashing """
    unique_id = sha256(os.urandom(48)).hexdigest()[:24]
    return unique_id

import random
def word_len(example):
    # 计算单词数量
    lens = len(example.split())
    return lens


def add_ctxs_to_item(base_item, items,  data_index_1,data_index_2):
    """ Adds 'ctxs' key to a group of items and updates GPT_answer """
    ctxs = []
    questions = []
    questions_ctxs = []
    end_qa_pair= []
    answer_map = {}
    total_length = 0
    # Process each item to create questions, answers, and modify GPT_answer
    for idx, item in enumerate(items):
        context_id = generate_unique_string()
        question_id = generate_unique_string()
        # middle answer 
        if idx < 10:
            question_id = generate_unique_string()
            ctxs.append({
                "ID": context_id,
                "text": item['context']+f" ### Question:"+ item['question'],
                "type": 'mid',
                "Q_ID": question_id,
                "GPT_answer":item['GPT_answer'],
            }) 
            total_length += word_len(item['context']+item['question']+item['GPT_answer'])
        # last answer 
        elif idx < 20:
            end_qa_pair.append(f"\nQuestion {idx-9}: {item['question']}")
            #  insert_text = f"### Retrieval: The answer to question {question_index} can be found in ID {correct_answer_id}."
            end_qa_pair.append(modify_gpt_answer_end(item['GPT_answer'], context_id, idx-9))
            ctxs.append({
                "ID": context_id,
                "text": item['context'],
                "type": 'end',
            })
            total_length += word_len(item['context']+item['question']+item['GPT_answer']+f"### Retrieval: The answer to question can be found in ID")
        else:
            ctxs.append({
                "ID": context_id,
                "text": item['context'],
                "type": 'false'
            })
            total_length += word_len(item['context'])
    # Add random contexts from dataset
    data_index_1,total_length = add_contexts_Orca(dataset_1,ctxs,data_index_1, total_length)
    data_index_2,total_length = add_contexts(dataset_2, ctxs, data_index_2, total_length)
    print(total_length)
    # Shuffle the list of contexts to ensure randomness
    random.shuffle(ctxs)
    # Reconstruct the mapping of question IDs to answer IDs after shuffling
    for i, ctx in enumerate(ctxs):
        if ctx.get('type') == 'mid':

            insert_text = f"Please answer the question ID: {ctx['ID']}, ### Answer: "
            Answer = ctx['GPT_answer']
            
            questions_ctxs.append({
                "ID": ctx['Q_ID'],
                "text": insert_text + Answer,
                "type": 3 ,
                "location" : i+1
            })

    offset = 0
    for new_ctx in questions_ctxs:
        start_position = new_ctx['location'] + offset
        # 检查起始位置是否超出列表长度，如果超出，则设置插入位置为列表末尾
        if start_position >= len(ctxs):
            insert_position = len(ctxs)
        else:
            insert_position = random.randint(start_position, len(ctxs))
        ctxs.insert(insert_position, new_ctx)
        offset += 1  

    base_item['GPT_answer'] = '\n '.join(end_qa_pair)
    base_item['question'] = ' '.join(questions)
    return ctxs, data_index_1,data_index_2


def add_contexts(dataset, ctxs, data_index,total_length):
    """ Adds contexts from the dataset while checking the token conditions """
    total_tokens = total_length
    max_tokens=19000
    while total_tokens < max_tokens:
        text = dataset[data_index]['text']
        data_index = data_index+1
        word_count = len(text.split())
        if 200 <= word_count <= 1000:
            context_id = generate_unique_string()
            ctxs.append({"ID": context_id, "text": text, "type": False})
            total_tokens += word_count  # Update the token count assuming 1 token per word
    return data_index,total_tokens

def add_contexts_Orca(dataset, ctxs, data_index,total_length):
    """ Adds contexts from the dataset while checking the token conditions """
    total_tokens = total_length
    num_QA = 0
    max_QA = 30
    while num_QA  < max_QA:
        num_QA += 1
        text = dataset[data_index]['question']
        response = dataset[data_index]['response']
   
        data_index = data_index+1
      
        question_id = generate_unique_string()
        context_id = generate_unique_string()
        ctxs.append({
                "ID":  context_id,
                "text": text,
                "type": 'mid',
                "Q_ID": question_id,
                "GPT_answer": response,
            })
        total_tokens += (word_len(text) + word_len(response))
   
        if total_tokens > 12000:
            print(num_QA)
            break
    return data_index,total_tokens

In [ ]:
def load_json_data(filepath):
    """ Loads JSON data from a file """
    try:
        with open(filepath, 'r') as file:
            data = json.load(file)
            return data
    except Exception as e:
        print(f"Error loading JSON data from {filepath}: {e}")
        return []
    
def expand_and_enhance_data(data, data_index_1,data_index_2 ):
    """ Expands each item by 32 times and enhances with ctxs """
    enhanced_data = []
    for item in data:
        for _ in range(1):  # Duplicate each item 32 times
            # Select 8 other random items
            other_items = random.sample(data, 29)
            other_items.insert(0, item)  # Include the current item as the first item
            new_item = item.copy()  # Make a copy of the item
            new_item['ctxs'],  data_index_1,data_index_2 = add_ctxs_to_item(new_item, other_items, data_index_1,data_index_2)
            enhanced_data.append(new_item)
    random.shuffle(enhanced_data)  # Shuffle all items to mix them up
    return enhanced_data, data_index_1,data_index_2

def save_data(data, output_file):
    """ Saves processed data into a JSON file """
    with open(output_file, 'w') as outfile:
        json.dump(data, outfile, indent=4)
    print(f"Data saved to {output_file}")

# Main logic
merged_file_path = 'merged_data.json'
output_file = 'version_6_new_4k.json'
merged_data = load_json_data(merged_file_path)
merged_data = merged_data[20:4020]

if merged_data:
    data_index_1 = 0
    data_index_2 = 0
    enhanced_data,data_index_1,data_index_2 = expand_and_enhance_data(merged_data, data_index_1,data_index_2)
    save_data(enhanced_data, output_file)
    print(data_index_1,data_index_2)

else:
    print("Failed to load data or context data.")

In [ ]:
#变为行
import json

# 打开并读取JSON文件
with open('version_6_4K_new.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 检查data是否是列表类型，如果是，按行写入每个元素
if isinstance(data, list):
    with open('Version_2.0/version6_no_orca_4K.json', 'w', encoding='utf-8') as output:
        for item in data:
            json.dump(item, output)
            output.write('\n')
else:
    print("JSON root is not a list. Cannot save items line by line.")

# Version_10_end_version?

In [ ]:
import json
import os
import random
from hashlib import sha256
from datasets import load_dataset

dataset_1 = load_dataset("Open-Orca/OpenOrca")

dataset_2 = load_dataset("DKYoon/SlimPajama-6B", split='train')

In [ ]:
from datasets import load_dataset

# 加载数据集
dataset_1 = load_dataset("Open-Orca/OpenOrca")

def filter_criteria(example):
    # 计算单词数量
    question_words = len(example['question'].split())
    response_words = len(example['response'].split())
    
    # 将question转换为小写并检查是否含有单词"translate"或"translation"
    question_lower = example['question'].lower()
    contains_translate = "translate" in question_lower or "translation" in question_lower
    
    # 判断单词数量是否都小于1000且question中不含"translate"和"translation"
    return question_words < 1000 and response_words < 1000 and not contains_translate

# 应用过滤条件
filtered_dataset = dataset_1.filter(filter_criteria)

# 随机打乱数据集
shuffled_dataset = filtered_dataset.shuffle()
dataset_2 = dataset_2.shuffle()
# 取出训练集
filtered_dataset = shuffled_dataset['train']

# 检查过滤后的数据
print(filtered_dataset)


In [ ]:

def modify_gpt_answer_end(gpt_answer, correct_answer_id, question_index):
    """ Modifies the GPT_answer to include the correct answer ID at the appropriate position """
    insert_text = f"### Retrieval: The answer to question {question_index} can be found in ID {correct_answer_id}."
    analysis_index = gpt_answer.find("### Analysis:")
    if analysis_index != -1:
        part_before = gpt_answer[:analysis_index + len("### Analysis:")]
        part_after = gpt_answer[analysis_index + len("### Analysis:"):]
        return  insert_text + part_before  + part_after
    else:
        return  insert_text  + gpt_answer 

def generate_unique_string():
    """ Generate a unique string based on hashing """
    unique_id = sha256(os.urandom(48)).hexdigest()[:24]
    return unique_id

import random

def word_len(example):
    # 计算单词数量
    lens = len(example.split())
    return lens
  


def add_ctxs_to_item(base_item, items,data_index_slimpajama,data_index_Orca):
    """ Adds 'ctxs' key to a group of items and updates GPT_answer """
    ctxs = []
    questions = []
    response_ctxs = []
    end_qa_pair= []
    total_length = 0
    answer_map = {}
    # Process each item to create questions, answers, and modify GPT_answer
    for idx, item in enumerate(items):
        context_id = generate_unique_string()
        question_id = generate_unique_string()

        # middle answer 
        if idx < 5:
            question_id = generate_unique_string()
            ctxs.append({
                "ID": context_id,
                "text": item['context']+f"### Question: "+item['question'],
                "type": 'mid',
                "Q_ID": question_id,
                "GPT_answer":item['GPT_answer'],
            })
            total_length += word_len(item['context']+f"### Question: "+item['question']) + word_len(item['GPT_answer'])
        # last answer 
        elif idx < 10:
            end_qa_pair.append(f"\nQuestion {idx-4}: {item['question']}")
            end_qa_pair.append(modify_gpt_answer_end(item['GPT_answer'], context_id, idx-4))
            ctxs.append({
                "ID": context_id,
                "text": item['context'],
                "type": 'end',
                "GPT_answer":item['GPT_answer'],
            })
            total_length += word_len(item['context']+f"### Question: "+item['question']) + word_len(item['GPT_answer'])
        else:
            ctxs.append({
                "ID": context_id,
                "text": item['context'],
                "type": 'false'
            })
            total_length += word_len(item['context']) 
    # Add random contexts from dataset
    data_index_Orca,total_length= add_contexts_Orca(filtered_dataset,ctxs,data_index_Orca,total_length)
    data_index_slimpajama,total_length= add_contexts(dataset_2, ctxs, data_index_slimpajama,total_length)
    # Shuffle the list of contexts to ensure randomness
    random.shuffle(ctxs)
    # Reconstruct the mapping of question IDs to answer IDs after shuffling
    for i, ctx in enumerate(ctxs):
        if ctx.get('type') == 'mid':

            insert_text = f"Please answer the question ID: {ctx['ID']}, ### Answer: "
            response_id =generate_unique_string()
            response_ctxs.append({
                "ID": response_id,
                "text": insert_text + ctx['GPT_answer'],
                "type": 3 ,
                "location" : i+1
            })
    offset = 0
    for new_ctx in response_ctxs:
        start_position = new_ctx['location'] + offset
        # 检查起始位置是否超出列表长度，如果超出，则设置插入位置为列表末尾
        if start_position >= len(ctxs):
            insert_position = len(ctxs)
        else:
            insert_position = random.randint(start_position, len(ctxs))
        ctxs.insert(insert_position, new_ctx)
        offset += 1  

    base_item['GPT_answer'] = '\n '.join(end_qa_pair)
    base_item['question'] = ' '.join(questions)
    return ctxs, data_index_slimpajama,data_index_Orca


def add_contexts(dataset, ctxs, data_index,total_length):
    """ Adds contexts from the dataset while checking the token conditions """
    total_tokens = total_length
    max_tokens=16000
    while total_tokens < max_tokens:
        text = dataset[data_index]['text']
        data_index = data_index+1
        word_count = len(text.split())
        if 200 <= word_count <= 1000:
            context_id = generate_unique_string()
            ctxs.append({"ID": context_id, "text": text, "type": False})
            total_tokens += word_count  # Update the token count assuming 1 token per word
    return data_index,total_tokens


def add_contexts_Orca(dataset, ctxs, data_index,total_length):
    """ Adds contexts from the dataset while checking the token conditions """
    total_tokens = total_length
    num_QA = 0
    max_QA = 500
    while num_QA  < max_QA:
        num_QA += 1
        text = dataset[data_index]['question']
        
        response = dataset[data_index]['response']
        data_index = data_index+1
        question_id = generate_unique_string()
        
        ctxs.append({
                "ID":  question_id,
                "text": text,
                "type": 'mid',
                "Q_ID": question_id,
                "GPT_answer": response,
            })
        total_tokens += word_len(text) + len(response)
        if total_tokens > 12000:
            print(num_QA)
            break
    return data_index,total_tokens



In [ ]:
def load_json_data(filepath):
    """ Loads JSON data from a file """
    try:
        with open(filepath, 'r') as file:
            data = json.load(file)
            return data
    except Exception as e:
        print(f"Error loading JSON data from {filepath}: {e}")
        return []
    
def expand_and_enhance_data(data,data_index_slimpajama,data_index_Orca):
    """ Expands each item by 32 times and enhances with ctxs """
    enhanced_data = []
    for item in data:
        for _ in range(1):  # Duplicate each item 32 times
            # Select 8 other random items
            other_items = random.sample(data, 11)
            other_items.insert(0, item)  # Include the current item as the first item
            new_item = item.copy()  # Make a copy of the item
            new_item['ctxs'], data_index_slimpajama,data_index_Orca = add_ctxs_to_item(new_item, other_items,data_index_slimpajama,data_index_Orca)
            enhanced_data.append(new_item)
    random.shuffle(enhanced_data)  # Shuffle all items to mix them up
    print(data_index_slimpajama,data_index_Orca)
    return enhanced_data

import json

def save_data(data, output_file):
    """Saves processed data into a JSON file, each list element on a new line if data is a list"""
    if isinstance(data, list):
        with open(output_file, 'w', encoding='utf-8') as output:
            for item in data:
                # 转换每个元素为 JSON 字符串并写入文件，每个元素后加一个换行符
                json.dump(item, output, ensure_ascii=False)
                output.write('\n')
        print(f"Data saved to {output_file}")
    else:
        print("JSON root is not a list. Cannot save items line by line.")


# Main logic
merged_file_path = 'merged_data.json'
output_file = 'version_11_test_4k.json'
merged_data = load_json_data(merged_file_path)
merged_data = merged_data[20:4020]

if merged_data:
    data_index_slimpajama = 0
    data_index_Orca =0 
    enhanced_data= expand_and_enhance_data(merged_data,data_index_slimpajama,data_index_Orca)
    save_data(enhanced_data, output_file)
   
else:
    print("Failed to load data or context data.")

# Version_12 End_version！！！

In [ ]:
import json
import os
import random
from hashlib import sha256
from datasets import load_dataset

dataset_1 = load_dataset("Open-Orca/OpenOrca")

dataset_2 = load_dataset("DKYoon/SlimPajama-6B", split='train')

In [ ]:


def filter_criteria(example):
    # 计算单词数量
    question_words = len(example['question'].split())
    response_words = len(example['response'].split())
    
    # 将question转换为小写并检查是否含有单词"translate"或"translation"
    # question_lower = example['question'].lower()
    # contains_translate = "translate" in question_lower or "translation" in question_lower
    id_starts_with_cot = example['id'].startswith('cot')
    # 判断单词数量是否都小于1000且question中不含"translate"和"translation"
    # return 80 < question_words < 1000 and 80 < response_words < 1000 and not contains_translate and id_starts_with_cot
    return  id_starts_with_cot and 20 < response_words and 20 < question_words

# 应用过滤条件
dataset_1 = dataset_1.filter(filter_criteria)

# 随机打乱数据集
dataset_1= dataset_1.shuffle()
dataset_2 = dataset_2.shuffle()
# 取出训练集
filtered_dataset = dataset_1['train']
dataset_1 = filtered_dataset
# 检查过滤后的数据



In [ ]:



def generate_unique_string():
    """ Generate a unique string based on hashing """
    unique_id = sha256(os.urandom(48)).hexdigest()[:24]
    return unique_id

import random

def word_len(example):
    # 计算单词数量
    lens = len(example.split())
    return lens
  


def add_ctxs_to_item(base_item, items,data_index_slimpajama,data_index_Orca,length):
    """ Adds 'ctxs' key to a group of items and updates GPT_answer """
    ctxs = []
    questions = []
    response_ctxs = []
    end_qa_pair= []
    total_length = 0
    answer_map = {}
    # Process each item to create questions, answers, and modify GPT_answer
    for idx, item in enumerate(items):
        question_id = generate_unique_string()
        ctxs.append({
                "ID": question_id,
                "text": item['context']+f" ### Question: "+item['question'],
                "type": 'question',
                "Q_ID": question_id,
                "GPT_answer":item['GPT_answer'],
            })
        total_length += word_len(item['context']+f" ### Question: "+item['question']+item['GPT_answer'])
    data_index_Orca,total_length= add_contexts_Orca(dataset_1,ctxs,data_index_Orca,total_length,length)
    data_index_slimpajama,total_length= add_contexts(dataset_2, ctxs, data_index_slimpajama,total_length,length)
    # print(total_length)
    random.shuffle(ctxs)
    # Reconstruct the mapping of question IDs to answer IDs after shuffling
    for i, ctx in enumerate(ctxs):
        if ctx.get('type') == 'question':
            questions.append(ctx['ID'])
            end_qa_pair.append(f"Answer question {ctx['ID']}: ")
            end_qa_pair.append(ctx['GPT_answer'])
    random.shuffle(ctxs)
    base_item['GPT_answer'] = '\n '.join(end_qa_pair)
    base_item['question'] = ', '.join(questions)
    return ctxs, data_index_slimpajama,data_index_Orca


def add_contexts(dataset, ctxs, data_index,total_length,length):
    """ Adds contexts from the dataset while checking the token conditions """
    total_tokens = total_length
    max_tokens=length
    while total_tokens < max_tokens:
        text = dataset[data_index]['text']
        data_index = data_index+1
        word_count = len(text.split())
        if 200 <= word_count <= 500:
            context_id = generate_unique_string()
            ctxs.append({"ID": context_id, "text": text, "type": False})
            total_tokens += word_len(text)  # Update the token count assuming 1 token per word
    return data_index,total_tokens


def add_contexts_Orca(dataset, ctxs, data_index,total_length,length):
    """ Adds contexts from the dataset while checking the token conditions """
    total_tokens = total_length
    num_QA = 0
    max_QA = 30
    while num_QA  < max_QA:
        num_QA += 1
        text = dataset[data_index]['question']
        response = dataset[data_index]['response']
        data_index = data_index+1
        question_id = generate_unique_string()
        
        ctxs.append({
                "ID":  question_id,
                "text": text,
                "type": 'question',
                "Q_ID": question_id,
                "GPT_answer": response,
            })
        total_tokens += (word_len(text) + word_len(response))
        if total_tokens > length:
            print(num_QA)
            break
    return data_index,total_tokens



In [ ]:
def load_json_data(filepath):
    """ Loads JSON data from a file """
    try:
        with open(filepath, 'r') as file:
            data = json.load(file)
            return data
    except Exception as e:
        print(f"Error loading JSON data from {filepath}: {e}")
        return []
    
def expand_and_enhance_data(data,data_index_slimpajama,data_index_Orca,length):
    """ Expands each item by 32 times and enhances with ctxs """
    enhanced_data = []
    for item in data:
        for _ in range(1):  # Duplicate each item 32 times
            # Select 8 other random items
            other_items = random.sample(data, 20)
            other_items.insert(0, item)  # Include the current item as the first item
            new_item = item.copy()  # Make a copy of the item
            new_item['ctxs'], data_index_slimpajama, data_index_Orca = add_ctxs_to_item(new_item, other_items,data_index_slimpajama,data_index_Orca,length)
            enhanced_data.append(new_item)
    random.shuffle(enhanced_data)  # Shuffle all items to mix them up
    return enhanced_data,data_index_slimpajama,data_index_Orca

import json
import os

def save_data(data, output_file, suffix):
    """Saves processed data into a JSON file, each list element on a new line if data is a list"""
    # 确保输出目录存在，如果不存在则创建
    os.makedirs(output_file, exist_ok=True)
    
    # 构建文件名，包括路径、基本文件名加上suffix后缀，以及文件扩展名
    filename = f"{output_file}data_{suffix}.json"
    
    if isinstance(data, list):
        with open(filename, 'w', encoding='utf-8') as output:
            for item in data:
                # 转换每个元素为 JSON 字符串并写入文件，每个元素后加一个换行符
                json.dump(item, output, ensure_ascii=False)
                output.write('\n')
        print(f"Data saved to {filename}")
    else:
        print("JSON root is not a list. Cannot save items line by line.")

# Main logic
merged_file_path = 'merged_data.json'
output_file = 'version_12_test_4k/'
merged_data = load_json_data(merged_file_path)
merged_data = merged_data[20:4020]

if merged_data:
    data_index_slimpajama = 0
    data_index_Orca =0 
    
    # 16 K
    length = 20000
    enhanced_data,data_index_slimpajama,data_index_Orca= expand_and_enhance_data(merged_data,data_index_slimpajama,data_index_Orca,length)
    print(data_index_slimpajama,data_index_Orca)
    save_data(enhanced_data, output_file,length)
    # # 8 K
    # length = length * 2
    # enhanced_data,data_index_slimpajama,data_index_Orca= expand_and_enhance_data(merged_data,data_index_slimpajama,data_index_Orca,length)
    # print(data_index_slimpajama,data_index_Orca)
    # save_data(enhanced_data, output_file,length)
    # # 16k
    # length = length * 2 + 1100
    # enhanced_data,data_index_slimpajama,data_index_Orca= expand_and_enhance_data(merged_data,data_index_slimpajama,data_index_Orca,length)
    # print(data_index_slimpajama,data_index_Orca)
    # save_data(enhanced_data, output_file,length)
    # # 32K
    # length = length * 2
    # enhanced_data = expand_and_enhance_data(merged_data,data_index_slimpajama,data_index_Orca,length)
    # save_data(enhanced_data, output_file,length)


   
else:
    print("Failed to load data or context data.")

265286 orcca orca 384W ->40K?

# tokenizer

In [ ]:
import os
import pandas as pd
from hashlib import sha256
from datasets import load_dataset
from transformers import LlamaTokenizer
from multiprocessing import Pool


def tokenize_data(text_id):
    """Tokenize text data and generate ID."""
    text, tokenizer = text_id
    tokenized_text = tokenizer(text, truncation=True, max_length=1000)
    # Convert BatchEncoding to dictionary and extract necessary fields
    tokenized_text_data = {
        'input_ids': tokenized_text['input_ids'],

    }
    return tokenized_text_data

def filter_by_word_count(example):
    """Calculate the number of words in the text."""
    word_count = len(example['text'].split())
    return 50 < word_count < 800

def main():
    # Load dataset
    dataset = load_dataset("DKYoon/SlimPajama-6B", split='train')
    # Take the first 1 million rows that meet the word count criteria
    num = 1000000
    dataset = dataset.select(range(num)).filter(filter_by_word_count)
    print(len(dataset))
    # Load tokenizer
    tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

    # Prepare data for multiprocessing
    texts = [(text, tokenizer) for text in dataset['text']]  # Assuming 'text' field exists

    # Use multiprocessing to tokenize data
    with Pool(os.cpu_count()) as pool:
        results = pool.map(tokenize_data, texts)

    # Convert results to DataFrame and save as Parquet file
    df = pd.DataFrame(results)
    df.to_parquet('tokenized_data_SlimPajama_Top1M.parquet')

    print("Data tokenized and saved successfully as Parquet.")

if __name__ == "__main__":
    main()


In [ ]:
import os
import pandas as pd
from hashlib import sha256
from datasets import load_dataset
from transformers import LlamaTokenizer
from multiprocessing import Pool

def filter_criteria(example):
    # 计算单词数量
    question_words = len(example['question'].split())
    response_words = len(example['response'].split())
    # 检查 id 是否以 'cot' 开头
    question_lower = example['question'].lower()
    contains_translate = "translate" in question_lower or "translation" in question_lower
    
    # 判断单词数量是否都小于1000且question中不含"translate"和"translation"
    return 80 < question_words < 800 and 100 < response_words < 800 and not contains_translate


def tokenize_data(example, tokenizer):
    """Tokenize question and response data."""
    tokenized_question = tokenizer(example['question'])
    tokenized_response = tokenizer(example['response'])
    return {
        'question_input_ids': tokenized_question['input_ids'],
        'response_input_ids': tokenized_response['input_ids']
    }

def main():
    # Load dataset
    dataset = load_dataset("Open-Orca/OpenOrca", split='train')
    # Filter dataset based on criteria
    filtered_dataset = dataset.filter(filter_criteria)
    print(filtered_dataset)

    # Load tokenizer
    tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

    # Prepare data for multiprocessing
    pool = Pool(os.cpu_count())
    results = [pool.apply_async(tokenize_data, (example, tokenizer)) for example in filtered_dataset]
    results = [result.get() for result in results]
    pool.close()
    pool.join()

    # Convert results to DataFrame and save as Parquet file
    df = pd.DataFrame(results)
    df.to_parquet('tokenized_data_Orca_all.parquet')

    print("Data tokenized and saved successfully as Parquet.")

if __name__ == "__main__":
    main()


# token dataset version 6

In [25]:
from datasets import load_dataset

# 设置文件路径
file_path_1 = "tokenized_data_Orca_all.parquet"
file_path_2 = "tokenized_data_SlimPajama_Top1M.parquet"
# 加载数据集
dataset_Orca = load_dataset('parquet', data_files=file_path_1)
dataset_Slim = load_dataset('parquet', data_files=file_path_2)
# 查看数据集
dataset_Orca = dataset_Orca['train']
dataset_Slim = dataset_Slim['train']
dataset_Slim = dataset_Slim.shuffle()
dataset_Orca = dataset_Orca.shuffle()

In [26]:
print(dataset_Slim)


Dataset({
    features: ['input_ids'],
    num_rows: 751834
})


In [27]:
dataset_Orca

Dataset({
    features: ['question_input_ids', 'response_input_ids'],
    num_rows: 421456
})

In [28]:
type(dataset_Slim[1]['input_ids'])

list

In [29]:
import json
import os
import random
from hashlib import sha256
from datasets import load_dataset
from transformers import LlamaTokenizer

In [30]:



def generate_unique_string():
    """ Generate a unique string based on hashing """
    unique_id = sha256(os.urandom(48)).hexdigest()[:24]
    return unique_id

import random

def word_len(example):
    # 计算单词数量
    lens = len(example.split())
    return lens
  
def token(text,tokenizer):
    """Tokenize text data and generate ID."""
   
    tokenized_text = tokenizer(text, truncation=True, max_length=10000)
    
    return tokenized_text['input_ids']

def input_start(input,label,length,tokenizer):
    context = f"\n### Instruction: Please carefully read the text and answer the questions contained within.\n"
    context += f"### Context:\n"
    add_list = token(context,tokenizer)
    # print(type(add_list))
    # print(type(input))
    input = input + add_list
    label = label +  [-100] * len(add_list)
    length = length + len(add_list)
    return  input, label, length 

def add_ctxs_to_item(items,data_index_slimpajama,data_index_Orca,length):
    """ Adds 'ctxs' key to a group of items and updates GPT_answer """
    ctxs = []

    questions_ctxs =[]
  
    total_length = 0
    Slim_length = []
    Orca_Q_length = []
    Orca_A_length = []
    Squad_Q_length = []
    Squad_A_length = []
    input_ids =[]
    labels = []
    tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
    input_ids,labels,total_length = input_start(input_ids,labels,total_length,tokenizer)

    # Process each item to create questions, answers, and modify GPT_answer
    for idx, item in enumerate(items):
        # mid
        if idx < 6:
            context_id = generate_unique_string()
            question_id = generate_unique_string()
            c_id = token(f"\nID: {context_id}, ",tokenizer)
            q_id = token(f"\nID: {question_id}, Answer question {context_id}: ",tokenizer)
            text = token(item['context']+f" ### Question: "+item['question'],tokenizer)
            GPT_answer = token(item['GPT_answer'],tokenizer)
            ctxs.append({
                    "ID": c_id+text,
                     #"text": text,
                    "type": 'question',
                    "Q_ID": q_id,
                    "GPT_answer":GPT_answer,
                })
            total_length += len(q_id+c_id+text+GPT_answer)
            Squad_Q_length += [len(text)]
            Squad_A_length += [len(GPT_answer)]
        #end 
        elif idx < 12:
            context_id = generate_unique_string()
            question_id = generate_unique_string()
            c_id = token(f"\nID: {context_id}, ",tokenizer)
            q_id = token(f"\nQuestion {idx-5}: {item['question']}\n Retrieval: The answer to question {idx-5} can be found in ID {context_id}.", tokenizer)
            text = token(item['context']+f" ### Question: "+item['question'],tokenizer)
            GPT_answer = token(item['GPT_answer'],tokenizer)
            ctxs.append({
                    "ID": c_id+text,
                    # "text": text,
                    "type": 'end',
                    "Q_ID": q_id+GPT_answer,
                    #"GPT_answer":GPT_answer,
                })
            total_length += len(q_id+c_id+text+GPT_answer)
            Squad_Q_length += [len(text)]
            Squad_A_length += [len(GPT_answer)]
        else:
            context_id = generate_unique_string()
            c_id = token(f"\nID: {context_id}, ",tokenizer)
            text = token(item['context']+f" ### Question: "+item['question'],tokenizer)
            ctxs.append({
                    "ID": c_id+text,
                    "type": False,
                })
            total_length += len(c_id+text)
            Squad_Q_length += [len(text)]

    data_index_Orca,total_length= add_contexts_Orca(dataset_Orca,ctxs,data_index_Orca,total_length,tokenizer,Orca_Q_length,Orca_A_length)


    ## context end:
    Instruction = token(f"\n Provide a detailed analysis based on the Context and offer your answers. Question: ",tokenizer)
    total_length += len(Instruction)    

    data_index_slimpajama,total_length= add_contexts(dataset_Slim, ctxs, data_index_slimpajama,total_length,length,tokenizer)


    random.shuffle(ctxs)
    # Reconstruct the mapping of question IDs to answer IDs after shuffling
 
    for i, ctx in enumerate(ctxs):
        if ctx.get('type') == 'question':
            questions_ctxs.append({
                "ID": ctx['Q_ID'],
                "text": ctx['GPT_answer'],
                "type": 'Answer',
                "location" : i+1
            })

    offset = 0
    random.shuffle(questions_ctxs)
    for new_ctx in questions_ctxs:
        start_position = new_ctx['location'] + offset
        # 检查起始位置是否超出列表长度，如果超出，则设置插入位置为列表末尾
        if start_position >= len(ctxs):
            insert_position = len(ctxs)
        else:
            insert_position = random.randint(start_position, len(ctxs))
        ctxs.insert(insert_position, new_ctx)
        offset += 1  

    # get end_data
    for item in ctxs:
        if item.get('type') == 'Answer':
            input_ids += item['ID']
            labels += [-100] * len(item['ID'])
            input_ids += item['text']
            labels += item['text']
        if item.get('type') == False or item.get('type') == 'question' or item.get('type') == 'end':
            input_ids += item['ID']
            labels += [-100] * len(item['ID'])
    
    # print("=="*100)
    # print('input len:',len(input_ids))
    # print('labels len:',len(labels))
    # print('length: ',total_length)

    input_ids += Instruction 
    labels += [-100] * len(Instruction)
  
    
    for item in ctxs:
        if item.get('type') == 'end':
            input_ids += item['Q_ID']
            labels += item['Q_ID']

    # print("=="*100)
    # print('input len:',len(input_ids))
    # print('labels len:',len(labels))
    # print('length: ',total_length)
    return {'input_ids':input_ids,'labels':labels}, {'Orca_Q':Orca_Q_length,'Orca_A':Orca_A_length,'Squad_Q': Squad_Q_length,'Squad_A':Squad_A_length},data_index_slimpajama,data_index_Orca


def add_contexts(dataset, ctxs, data_index,total_length,length,tokenizer):
    """ Adds contexts from the dataset while checking the token conditions """
    total_tokens = total_length
    max_tokens= length
    while total_tokens < max_tokens:
        text = dataset[data_index]['input_ids']
        data_index = data_index+1
        context_id = generate_unique_string()
        c_id = token(f"\nID: {context_id}, ",tokenizer)
        total_tokens += len(c_id+text)
        if (total_tokens <= max_tokens):
            ctxs.append({"ID": c_id+text,"type": False})
        else:
            all = c_id + text
            ctxs.append({"ID": all[:max_tokens-total_tokens], "type": False})
            total_tokens = 32768
          # Update the token count assuming 1 token per word
    return data_index,total_tokens


def add_contexts_Orca(dataset, ctxs, data_index,total_length,tokenizer,Orca_Q_length,Orca_A_length
    ):
    """ Adds contexts from the dataset while checking the token conditions """

    # features: ['question_input_ids', 'response_input_ids'],
    total_tokens = total_length
    num_QA = 0
    max_QA = 80
    while num_QA  < max_QA:
        num_QA += 1
        context_id = generate_unique_string()
        question_id = generate_unique_string()
        text = dataset[data_index]['question_input_ids']
        response = dataset[data_index]['response_input_ids']
        data_index += 1
        c_id = token(f"\nID: {context_id}, ",tokenizer)
        q_id = token(f"\nID: {question_id}, ",tokenizer)  
        q_start = token(f" Answer question {context_id}: ",tokenizer)  
        ctxs.append({
                "ID":  c_id+text,
                # "text": text,
                "type": 'question',
                "Q_ID": q_id+q_start,
                "GPT_answer":response,
            })
        total_tokens += len(c_id+text+q_id+q_start+response)
        Orca_Q_length += [len(text)]
        Orca_A_length += [len(response)]

        if total_tokens > 24000:
            # print(num_QA)
            break
    return data_index,total_tokens



In [32]:
import json
import os
import random
from multiprocessing import Pool

def load_json_data(filepath):
    """ Loads JSON data from a file """
    try:
        with open(filepath, 'r') as file:
            data = json.load(file)
            return data
    except Exception as e:
        print(f"Error loading JSON data from {filepath}: {e}")
        return []
    

def expand_and_enhance_data(data,data_index_slimpajama,data_index_Orca,length):
    """ Expands each item by 32 times and enhances with ctxs """
    enhanced_data = []
    length_distri = []
    for item in data:
        for _ in range(4):  # Duplicate each item 32 times
            # Select 8 other random items
            other_items = random.sample(data, 14)
            other_items.insert(0, item)  # Include the current item as the first item
            new_item = item.copy()  # Make a copy of the item
            end_data, length_distribution, data_index_slimpajama, data_index_Orca = add_ctxs_to_item(other_items,data_index_slimpajama,data_index_Orca,length)
            enhanced_data.append(end_data)
            length_distri.append(length_distribution)
    random.shuffle(enhanced_data)  # Shuffle all items to mix them up
    return enhanced_data,length_distri,data_index_slimpajama,data_index_Orca



def save_data_1(data, output_file):
    """ Save the data into a parquet file """
    df = pd.DataFrame(data, columns=['input_ids', 'labels'])
    df.to_parquet(output_file, index=False)
    print(f"Data saved to {output_file}")

def save_data_2(data, output_file):
    """ Save the data into a parquet file """
    df = pd.DataFrame(data, columns=['Orca_Q','Orca_A','Squad_Q','Squad_A'])
    df.to_parquet(output_file, index=False)
    print(f"Data saved to {output_file}")
    


# Main logic
merged_file_path = 'merged_data.json'
output_file = 'token_version_6_32K_version_5.parquet'
output_file_2 = 'distribution.parquet'
merged_data = load_json_data(merged_file_path)
merged_data = merged_data[20:4020]

if merged_data:
    data_index_slimpajama = 0
    data_index_Orca =0 
    
    # 16 K
    length = 32768
    enhanced_data,length_distri,data_index_slimpajama,data_index_Orca= expand_and_enhance_data(merged_data,data_index_slimpajama,data_index_Orca,length)
    print(data_index_slimpajama,data_index_Orca)
    save_data_1(enhanced_data, output_file)
    save_data_2(length_distri, output_file_2)
else:
    print("Failed to load data or context data.")

1376 1973
Data saved to token_version_6_32K_version_5.parquet
Data saved to distribution.parquet


In [14]:
import json
import os
import random
from hashlib import sha256
from datasets import load_dataset
from transformers import LlamaTokenizer

file_path_1 = "distribution.parquet"

# 加载数据集
dataset= load_dataset('parquet', data_files=file_path_1)

Generating train split: 80 examples [00:00, 31530.19 examples/s]


{'Orca_Q': [286,
  199,
  406,
  211,
  315,
  198,
  205,
  801,
  630,
  240,
  459,
  239,
  134,
  207,
  260,
  308,
  440,
  553,
  209,
  222,
  288,
  124,
  1000,
  1000,
  183,
  474,
  224,
  612,
  553],
 'Orca_A': [180,
  140,
  144,
  142,
  198,
  216,
  177,
  199,
  213,
  234,
  161,
  124,
  263,
  171,
  288,
  145,
  204,
  131,
  159,
  294,
  224,
  316,
  176,
  324,
  206,
  178,
  315,
  155,
  192],
 'Squad_Q': [190,
  153,
  244,
  265,
  115,
  250,
  169,
  182,
  247,
  281,
  190,
  192,
  277,
  257,
  252],
 'Squad_A': [210, 74, 190, 208, 245, 196, 119, 241, 154, 186]}

In [17]:
for i in range(4000):
    input_len = len(dataset['train'][i]['input_ids'])
    label_len = len(dataset['train'][i]['labels'])
    if input_len != 32768:
        print("len:",input_len)
        print(i)
    if label_len != 32768:
        print("len:",label_len)
    

In [6]:
from datasets import load_dataset
import pandas as pd
from itertools import chain


file_path_1 = "/home/yuhao/gsm8k-ScRel/data_Context/context_data/tokenized_data_Orca_all.parquet"

# 加载数据集
dataset= load_dataset('parquet', data_files=file_path_1)

In [7]:
dataset = dataset['train']
concatenated_ids = [q + r for q, r in zip(dataset['question_input_ids'], dataset['response_input_ids'])]

# Flatten the list of IDs and then concatenate multiple entries together
all_input_ids = list(chain(*concatenated_ids))

# Split the long sequence into chunks of 32768 tokens each
chunks = [all_input_ids[i:i + 32769] for i in range(0, len(all_input_ids), 32769)]

# If the last chunk is not full, you can choose to discard it
if len(chunks[-1]) != 32769:
    chunks.pop()  # This removes the last incomplete chunk

# Create a DataFrame
df = pd.DataFrame({'input_ids': chunks})

# Save to a Parquet file
df.to_parquet('short_Orca.parquet')

In [8]:
file_path_1 = "short_Orca.parquet"

# 加载数据集
dataset= load_dataset('parquet', data_files=file_path_1)
dataset

Generating train split: 8168 examples [00:03, 2154.56 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 8168
    })
})

In [25]:
from datasets import load_dataset
import pandas as pd
from itertools import chain
all_input_ids = list(chain(*dataset['input_ids']))

# 按每 32769 个元素分割长序列
chunks = [all_input_ids[i:i + 32769] for i in range(0, len(all_input_ids), 32769)]

# 如果最后一个 chunk 不满 32769，可以选择丢弃或处理它
if len(chunks[-1]) != 32769:
    chunks.pop()  # 如果你不想保留最后一个不完整的 chunk

# 创建 DataFrame
df = pd.DataFrame({'input_ids': chunks})

# 保存到 Parquet 文件
df.to_parquet('short_slim.parquet')


In [1]:
import json
import os
import random
from hashlib import sha256
from datasets import load_dataset
from transformers import LlamaTokenizer

file_path_1 = "/home/yuhao/gsm8k-ScRel/data_Context/context_data/tokenized_data_Orca_all.parquet"

# 加载数据集
dataset= load_dataset('parquet', data_files=file_path_1)

/home/yuhao/anaconda/envs/FILM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question_input_ids', 'response_input_ids'],
        num_rows: 421456
    })
})

In [30]:
for i in range(40):
    input_len = len(dataset['train'][i]['input_ids'])
    #label_len = len(dataset['train'][i]['labels'])
    if input_len != 32768:
        print("len:",input_len)
        print(i)
    
    

len: 32769
0
len: 32769
1
len: 32769
2
len: 32769
3
len: 32769
4
len: 32769
5
len: 32769
6
len: 32769
7
len: 32769
8
len: 32769
9
len: 32769
10
len: 32769
11
len: 32769
12
len: 32769
13
len: 32769
14
len: 32769
15
len: 32769
16
len: 32769
17
len: 32769
18
len: 32769
19
len: 32769
20
len: 32769
21
len: 32769
22
len: 32769
23
len: 32769
24
len: 32769
25
len: 32769
26
len: 32769
27
len: 32769
28
len: 32769
29
len: 32769
30
len: 32769
31
len: 32769
32
len: 32769
33
len: 32769
34
len: 32769
35
len: 32769
36
len: 32769
37
len: 32769
38
len: 32769
39
